<a href="https://colab.research.google.com/github/vibhuverma17/AUTOML/blob/main/Transfer_Learning_Image_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# TRANSFER LEARNING PROBLEM TARGET NEEDS TO BE VERY SIMILAR

import matplotlib.pylab as plt
import tensorflow as tf
import tensorflow_hub as hub

import os
import numpy as np

import tensorflow_datasets as tfds

import seaborn as sns

from sklearn.metrics import confusion_matrix

In [ ]:
tf.__version__

---
title: "Transfer Learning with Rock Paper Scissors Dataset"
output: html_document
---

## Overview

In this document, we will apply **transfer learning** to classify images of rock, paper, and scissors using a pre-trained model, **MobileNetV2**. Transfer learning allows us to take advantage of a model that has already learned from a large dataset and apply that knowledge to a smaller, related dataset.

## Steps

### 1. Load the Dataset

We load the **rock_paper_scissors** dataset from TensorFlow Datasets (TFDS). This dataset contains images labeled as rock, paper, or scissors. We split the data into training and testing sets.

### 2. Preprocess the Images

Before using the images in the model, we resize them to a size that matches what the pre-trained MobileNetV2 model expects (224x224 pixels). We also apply specific preprocessing steps to adjust the images for MobileNetV2.

### 3. Optimize the Data Pipeline

To make training faster and more efficient, we:
- **Cache** the processed images so they don’t need to be reloaded in each epoch.
- **Batch** the images into groups of 32, which allows the model to process them together.
- **Prefetch** the next batch while the model is processing the current one, reducing downtime during training.

### 4. Load a Pre-Trained Model

We use **MobileNetV2**, a model that has already been trained on a large dataset (ImageNet). This allows us to leverage the knowledge the model has learned, such as recognizing general patterns in images. We remove the final layer of MobileNetV2, as we will add our own for the classification task.

### 5. Build the Model

We add a **GlobalAveragePooling2D** layer to reduce the model’s output dimensions and a **Dense** layer with 3 units (one for each class: rock, paper, and scissors). The **softmax** activation function ensures the model outputs probabilities for each class.

### 6. Compile and Train the Model

We compile the model with the **Adam optimizer**, use the **sparse categorical cross-entropy loss** (because this is a multi-class classification problem), and track **accuracy** as a metric. Then, we train the model on the training dataset.

### 7. Fine-Tuning

After training the new layers, we unfreeze some layers of the pre-trained MobileNetV2 and allow them to adjust to the specific task of classifying rock, paper, and scissors. We then train the model again with a smaller learning rate to fine-tune it.

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

# Load the dataset
datasets, info = tfds.load(name='rock_paper_scissors', with_info=True, as_supervised=True, split=['train', 'test'])
train_ds, test_ds = datasets

# Preprocessing function
def preprocess(image, label):
    image = tf.image.resize(image, (224, 224))  # Resize images to match the input size of MobileNetV2
    image = tf.keras.applications.mobilenet_v2.preprocess_input(image)  # Preprocessing for MobileNetV2
    return image, label

# Split the train dataset into training and validation datasets (80% train, 20% validation)
val_size = int(0.2 * info.splits['train'].num_examples)  # 20% for validation
train_ds, val_ds = train_ds.take(info.splits['train'].num_examples - val_size), train_ds.skip(info.splits['train'].num_examples - val_size)

# Apply preprocessing with batch size and caching to training and validation sets
train_ds = (train_ds
            .map(preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE)  # Preprocessing
            .cache()  # Caching the data to avoid recomputing
            .batch(32)  # Set batch size
            .prefetch(tf.data.experimental.AUTOTUNE))  # Prefetch for better performance

val_ds = (val_ds
          .map(preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE)  # Preprocessing
          .cache()  # Caching the data
          .batch(32)  # Set batch size
          .prefetch(tf.data.experimental.AUTOTUNE))  # Prefetch

test_ds = (test_ds
           .map(preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE)  # Preprocessing
           .cache()  # Caching the data
           .batch(32)  # Set batch size
           .prefetch(tf.data.experimental.AUTOTUNE))  # Prefetch

In [ ]:
# Load a pre-trained MobileNetV2 model without the top (fully connected) layer
base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

# Freeze the entire base model to prevent its layers from being trained
base_model.trainable = False

# Build the model by adding a classification head on top of the pre-trained model
model = tf.keras.Sequential([
    base_model,  # Base pre-trained model
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(3, activation='softmax')  # Final classification layer for 3 classes
])

# Compile the model (only the new classification head will be trained)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

class CollectStatsCallback(tf.keras.callbacks.Callback):
    def __init__(self):
        self.batch_losses = []
        self.batch_accuracies = []

    def on_batch_end(self, batch, logs=None):
        self.batch_losses.append(logs['loss'])
        self.batch_accuracies.append(logs['accuracy'])

# Create the callback instance
collect_stats_callback = CollectStatsCallback()

# Train the model (only the classification head will be trained, base model is frozen)
model.fit(train_ds, epochs=2, validation_data=val_ds,callbacks=[collect_stats_callback])

In [ ]:
# Plot batch loss
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(collect_stats_callback.batch_losses, label='Batch Loss')
plt.title('Batch Loss During Training')
plt.xlabel('Batch')
plt.ylabel('Loss')
plt.legend()

# Plot batch accuracy
plt.subplot(1, 2, 2)
plt.plot(collect_stats_callback.batch_accuracies, label='Batch Accuracy')
plt.title('Batch Accuracy During Training')
plt.xlabel('Batch')
plt.ylabel('Accuracy')
plt.legend()

# Show the plots
plt.tight_layout()
plt.show()

In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_ds)

# Print the results
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')

In [ ]:
# Get predictions and true labels from the test dataset
y_true = []
y_pred = []

for image, label in test_ds:
    # Get the true labels
    y_true.append(label.numpy())

    # Get the predicted labels
    predictions = model.predict(image)
    predicted_labels = np.argmax(predictions, axis=1)
    y_pred.append(predicted_labels)

# Convert lists to arrays
y_true = np.concatenate(y_true)
y_pred = np.concatenate(y_pred)
# Generate confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Plot confusion matrix
plt.figure(figsize=(6, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Rock', 'Paper', 'Scissors'], yticklabels=['Rock', 'Paper', 'Scissors'])
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
## WE CAN DO QUANTIZATION OR TF LITE TO MAKE THIS EVEN LIGHTER
# QUANTIZATION CAN BE DONE SURING TRAINING OR POST MODEL